# MNIST Neural Network

Working from this [kernel in Kaggle](https://www.kaggle.com/sdelecourt/cnn-with-pytorch-for-mnist)

In [6]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split


In [11]:
# Trying to keep training parameters in the same place
BATCH_SIZE = 32
EPOCHS = 5

In [12]:
df = pd.read_csv('digit-recognizer/train.csv')
print(df.shape)

(42000, 785)


In [13]:
y = df['label'].values
X = df.drop(['label'],1).values # Drop the labels so you don't get data pollution

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15) # Split into test and training data

In [9]:
torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long; But why?

# create feature and targets tensor for test set
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

It looks like the data flow in torch in this example goes follows:
1. Read the data in from csv file
2. Convert the data to numpy array (can be combined with step 1)
3. Split the dataset into test and train data, using scikit learn functionality
4. Convert the test and train datasets into *Torch Tensor*
5. Convert the *Torch Tensors* into *Tensor Datasets* for both train and test data
6. Input the Datasets into a data loader, considering a specified batch size

Seems like an unnecessary number of steps; Chance to optimize, at the very least for shorter and clearer code.

In [10]:
# Same as other example; Create an inherited class for the neural network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,250)
        self.linear2 = nn.Linear(250,100)
        self.linear3 = nn.Linear(100,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = self.linear3(X)
        return F.log_softmax(X, dim=1)
 
mlp = MLP()
print(mlp)

MLP(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


'*' Copied from the example code

We have 784*(250+1) + 250*(100+1) + 100*(10+1) = 222 360 parameters to train

In [20]:
def fit(model, train_loader):
    # Adam is a method for stochastic gradient descent; avail. below
    # https://pytorch.org/docs/stable/_modules/torch/optim/adam.html
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss() # Cross Entropy loss is a loss fxn, AKA log loss
    # Read more aboout cross entropy, or log loss, here: https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html
    model.train() # Calling .train() method on nn.Module object
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                

Run the model

In [21]:
fit(mlp, train_loader)

Epoch : 0 [0/35700 (0%)]	Loss: 0.408600	 Accuracy:96.875%
Epoch : 0 [1600/35700 (4%)]	Loss: 0.005384	 Accuracy:98.468%
Epoch : 0 [3200/35700 (9%)]	Loss: 0.007809	 Accuracy:98.360%
Epoch : 0 [4800/35700 (13%)]	Loss: 0.011668	 Accuracy:98.489%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.001402	 Accuracy:98.243%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.048860	 Accuracy:98.095%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.158247	 Accuracy:98.100%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.060775	 Accuracy:98.015%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.024067	 Accuracy:97.927%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.252585	 Accuracy:97.894%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.000414	 Accuracy:97.917%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.000932	 Accuracy:97.902%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.172058	 Accuracy:97.889%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.089318	 Accuracy:97.883%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.000671	 Accuracy:97.838%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.001018	 Accuracy:97.828%
Epoch